In [1]:
# Logger
import logging
logging.basicConfig(level=logging.WARN)

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load PyALCS ACS2 module
from lcs.agents.acs2 import ACS2, Configuration
from bacs.agents.bacs.utils.GymMazeWrapper import _maze_metrics, parse_metrics_to_df, plot_performance

# Load environments
import gym
import my_mazes

# Allow to parallelize all benchmarks to do
import parmap

# For calculation standard deviation
import statistics

# Agent - ACS2 - BENCHMARKING

## Main Parameters

In [2]:
CLASSIFIER_LENGTH = 8
NUMBER_OF_POSSIBLE_ACTIONS = 8

NUMBER_OF_EXPLORE_TRIALS = 2000
DO_ACTION_PLANNING_EXPLORE = False
DO_GA_EXPLORE = False
DO_SUBSUMPTION_EXPLORE = True
BETA_EXPLORE = 0.025
EPSILON = 0.3

NUMBER_OF_EXPLOIT_TRIALS = 500
DO_ACTION_PLANNING_EXPLOIT = False
DO_SUBSUMPTION_EXPLOIT = True
BETA_EXPLOIT = 0.00

NUMBER_OF_ITERATIONS_TO_BENCH = 30

## Full Details of Agent Parameters

In [3]:
cfg_explore = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=DO_GA_EXPLORE,
    do_subsumption=DO_SUBSUMPTION_EXPLORE,
    do_action_planning=DO_ACTION_PLANNING_EXPLORE,
    action_planning_frequency=50,
    beta=BETA_EXPLORE,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=EPSILON,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8
)

cfg_exploit = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=False,
    do_subsumption=DO_SUBSUMPTION_EXPLOIT,
    do_action_planning=DO_ACTION_PLANNING_EXPLOIT,
    action_planning_frequency=50,
    beta=BETA_EXPLOIT,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=0.0,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8
)

### Benchmarking - Maze

In [4]:
def bench_on_maze(env,n):
    avg_explore_list = []
    avg_exploit_list = []
    
    for i in range(n):
        
        # Initialize environment
        maze = gym.make(env.id)
        
        # Reset it, by putting an agent into random position
        situation = maze.reset()
    
        # Training of ACS2 - Exploration
        agent_explore = ACS2(cfg_explore)
        population_explore, metrics_explore = agent_explore.explore(maze, NUMBER_OF_EXPLORE_TRIALS)
    
        # Using ACS2 - Exploitation
        agent_exploit = ACS2(cfg_exploit, population_explore)
        population_exploit, metrics_exploit = agent_exploit.exploit(maze, NUMBER_OF_EXPLOIT_TRIALS)

        # Plotting performance
        #metrics_df = parse_metrics_to_df(metrics_explore, metrics_exploit)
        #plot_performance(agent_exploit, maze, metrics_df, cfg_exploit, env.id)
        
        # Get average in exploration
        avg_step_explore = 0
        for trial in metrics_explore:
            avg_step_explore += trial['steps_in_trial']
        avg_step_explore /= NUMBER_OF_EXPLORE_TRIALS
        avg_explore_list.append(avg_step_explore)
        
        # Get average in exploitation
        avg_step_exploit = 0
        for trial in metrics_exploit:
            avg_step_exploit += trial['steps_in_trial']
        avg_step_exploit /= NUMBER_OF_EXPLOIT_TRIALS
        avg_exploit_list.append(avg_step_exploit)
        
        # Get other metrics
        knowledge_list.append(metrics_exploit[-1]['knowledge'])
        population_list.append(metrics_exploit[-1]['population'])
        numerosity_list.append(metrics_exploit[-1]['numerosity'])
        reliable_list.append(metrics_exploit[-1]['reliable'])
        
        print(env.id,'|','#' * (i+1) ,' ' * (n-(i+1)), '|')
    
    # Compute the means and standard deviations
    avg_explore = float(sum(avg_explore_list)) / n
    std_explore = statistics.pstdev(avg_explore_list)
    avg_exploit = float(sum(avg_exploit_list)) / n
    std_exploit = statistics.pstdev(avg_exploit_list)
    avg_knowledge = float(sum(knowledge_list)) / n
    std_knowledge = statistics.pstdev(knowledge_list)
    avg_population = float(sum(population_list)) / n
    std_population = statistics.pstdev(population_list)
    avg_numerosity = float(sum(numerosity_list)) / n
    std_numerosity = statistics.pstdev(numerosity_list)
    avg_reliable = float(sum(reliable_list)) / n
    std_reliable = statistics.pstdev(reliable_list)
    
    result = {
        'maze'             : env.id,
        'avg_explore'      : avg_explore,
        'std_explore'      : std_explore,
        'avg_exploit'      : avg_exploit,
        'std_exploit'      : std_exploit,
        'avg_knowledge'    : avg_knowledge,
        'std_knowledge'    : std_knowledge,
        'avg_population'   : avg_population,
        'std_population'   : std_population,
        'avg_numerosity'   : avg_numerosity,
        'std_numerosity'   : std_numerosity,
        'avg_reliable'     : avg_reliable,
        'std_reliable'     : std_reliable,
        'avg_exploit_list' : avg_exploit_list
    }
    
    print(result)
    
    return result

Custom functions for getting available environments in Gym depending on the type of mazes :

In [5]:
filter_envs_typeIII = lambda env: env.id.startswith("Maze10-") or env.id.startswith("MazeE") \
    or env.id.startswith("Woods10")

filter_envs_typeII = lambda env: env.id.startswith("MazeF4") or env.id.startswith("Maze7") \
    or env.id.startswith("Sutton") or env.id.startswith("MiyazakiB") \
    or env.id.startswith("Lab1")

filter_envs_typeI = lambda env: env.id.startswith("MazeB") or env.id.startswith("MazeD") \
    or env.id.startswith("Littman") or env.id.startswith("MiyazakiA") \
    or env.id.startswith("Cassandra")

filter_envs_na = lambda env: env.id.startswith("MazeF1") or env.id.startswith("MazeF2") \
    or env.id.startswith("MazeF3") or env.id.startswith("Woods1-") \
    or env.id.startswith("Woods14") or env.id.startswith("Maze4") \
    or env.id.startswith("Maze5") or env.id.startswith("MazeA")

all_envs = [env for env in gym.envs.registry.all()]

Benchmarking without the genetic algorithms on mazes :

In [6]:
#maze_envs = [env for env in all_envs if filter_envs_na(env) or filter_envs_typeI(env) or filter_envs_typeII(env) or filter_envs_typeIII(env)]
maze_envs = [env for env in all_envs if ("Woods1-")]

results = parmap.map(bench_on_maze, maze_envs, NUMBER_OF_ITERATIONS_TO_BENCH, pm_pbar=True, pm_processes=1)

888it [00:00, 14182.52it/s]            


AssertionError: 

Parsing the previous result to get a markdown string for each environment :

In [ ]:
markdown_str = ''
for item in results:
    markdown_str += '|' + item['maze'] + '|'
    markdown_str += "{:.2f}".format(item['avg_explore']) + '|'
    markdown_str += "{:.2f}".format(item['std_explore']) + '|'
    markdown_str += "{:.2f}".format(item['avg_exploit']) + '|'
    markdown_str += "{:.2f}".format(item['std_exploit']) + '|'
    markdown_str += "{:.2f}".format(min(item['avg_exploit_list'])) + '|'
    markdown_str += "{:.2f}".format(max(item['avg_exploit_list'])) + '|'
    markdown_str += '0' + '|'
    markdown_str += "{:.2f}".format(item['avg_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['std_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['avg_population']) + '|'
    markdown_str += "{:.2f}".format(item['std_population']) + '|'
    markdown_str += "{:.2f}".format(item['avg_reliable']) + '|'
    markdown_str += "{:.2f}".format(item['std_reliable']) + '|'
    markdown_str += '\n'
print(markdown_str)

### Last results
 
The parameters used are:
 
- no use of action planning
- use of subsumption in the anticipatory learning process
- use od subsumption in the genetic algorithm is this last one is used
- $\gamma$ = 0.95
- $\theta_i$ = 0.1
- $\theta_r$ = 0.9
- $u_{max}$ = 8 (length of the classifier)
- $\theta_{exp}$ = 20
- $\beta$ = 0.025
- $\epsilon$ = 0.3
 
If genetic algorithms are used:

- $\theta_{ga}$ = 100
- $\theta_{as}$ = 20
- $\mu$ = 0.3
- $\chi$ = 0.8,
 
 For 2000 trials in Exploration and 500 trials in Exploitation for all environments, computed 30 times for calculating all averages.
 
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|14.64|1.40|19.32|19.14|4.12|76.06|0|20.77|1.86|1676.53|136.91|69.30|7.76|
|Woods101demi-v0|10.41|1.60|20.34|11.51|3.44|43.52|5|50.24|1.78|464.90|41.69|30.63|2.89|
|Maze10-v0|53.48|1.47|76.63|10.97|53.01|95.21|0|65.53|1.68|157.00|13.62|34.70|3.48|
|Woods102-v0|12.28|3.51|18.44|12.34|3.34|40.71|6|43.17|4.38|563.43|51.84|37.00|4.02|
|Woods100-v0|3.91|0.23|2.34|0.05|2.26|2.43|30|60.33|1.80|23.53|2.35|6.43|0.56|
|Woods101-v0|10.85|0.83|23.92|21.08|3.65|71.55|2|62.47|2.66|125.00|15.22|22.93|1.53|
|MazeE1-v0|12.46|1.87|20.73|10.78|5.51|45.05|0|19.94|3.45|1898.33|173.83|82.53|18.32|
|**Type II**      ||||||||||||||
|MazeF4-v0|15.64|3.02|13.50|21.34|4.39|81.69|25|81.07|3.57|118.83|22.23|31.17|3.59|
|Maze7-v0|15.02|4.10|10.24|19.00|4.17|80.89|27|80.32|3.42|102.63|16.89|32.80|3.39|
|Sutton-v0|61.88|15.79|67.90|11.99|41.14|87.74|0|44.64|3.67|1949.47|133.86|226.83|28.77|
|Lab1-v0|99.33|0.68|99.99|0.05|99.74|100.00|0|62.43|2.38|990.27|74.10|221.43|13.34|
|MiyazakiB-v0|11.84|2.84|6.52|6.95|3.43|40.48|22|47.93|6.04|828.27|90.80|105.53|18.17|
|**Type I**       ||||||||||||||
|MazeB-v0|17.04|4.37|19.47|31.60|5.00|100.00|1|53.33|7.90|378.97|31.72|89.57|15.19|
|Littman89-v0|10.04|2.86|13.47|12.74|3.94|56.00|16|45.54|6.13|319.63|53.56|50.73|7.93|
|MiyazakiA-v0|7.54|1.26|4.21|1.17|3.14|8.38|26|31.31|4.99|970.13|131.49|70.83|14.27|
|MazeD-v0|10.15|1.87|4.63|0.80|4.00|7.00|16|44.58|6.63|288.03|27.39|62.37|11.43|
|Cassandra4x4-v0|5.35|0.69|3.30|0.88|2.25|7.48|9|31.85|4.92|602.57|67.08|46.93|10.94|
|Littman57-v0|7.42|1.40|4.29|0.57|3.61|5.41|30|62.85|6.19|95.23|24.01|24.63|2.69|
|**Not Aliased**||||||||||||||
|Maze4-v0|9.47|1.22|4.43|0.48|3.73|5.60|27|64.75|6.12|250.50|21.78|111.00|13.80|
|Maze5-v0|13.61|2.31|6.00|1.26|4.82|10.94|28|66.16|6.22|329.37|27.07|155.23|18.38|
|MazeA-v0|15.76|2.85|7.03|1.05|6.00|9.00|14|75.96|8.37|134.73|11.98|84.20|10.28|
|MazeF1-v0|3.67|0.61|2.31|0.45|1.75|3.09|26|92.14|5.93|14.30|0.53|13.13|0.76|
|MazeF2-v0|4.79|0.39|2.72|0.26|2.42|3.53|30|98.04|3.16|20.00|0.45|17.80|0.65|
|MazeF3-v0|6.44|0.64|3.52|0.22|3.29|4.27|30|97.62|2.95|32.03|2.71|27.83|1.34|
|Woods1-v0|3.25|0.60|2.09|0.36|1.60|3.35|11|59.55|11.22|68.33|7.58|29.07|6.21|
|Woods14-v0|33.00|6.94|9.54|0.22|9.08|10.03|30|100.00|0.00|39.17|2.02|39.17|2.02|


### Complexity and Optimal Performance by Maze

These measures come from Zatuchna and Bagnall paper :

> Zhanna V. Zatuchna and Anthony Bagnall. 2009. Learning Mazes with Aliasing States: An LCS Algorithm with Associative Perception. Adaptive Behavior - Animals, Animats, Software Agents, Robots, Adaptive Systems 17, 1 (February 2009), 28-57. DOI=http://dx.doi.org/10.1177/1059712308099230

$\phi$ is about the average distance to reward in maze. Its calculation depends mainly on the type of maze. Here is reported $\phi$' if the maze is aliased, the original $\phi$ otherwise.

$\psi$ is about the complexity of the maze that depends on the average distance to reward and on the average steps taken by trained Q-learning.

I also added $d$, the maximal distance to food I consider to count the number of succesfull tries.

The question marks (**?**) highlight some discrepancies present in the original paper.

The asterisks (**\***) indicate pieces of information were not provided.

For further details, please see the original paper.

#### For Type III Aliased Mazes

|       |MazeE2      |Woods101demi|Maze10      |Woods102    |Woods100    |Woods101    |MazeE1       |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:-----------:|
|$d$    |3           |4           |8           |4           |3           |4           |4            |
|$\phi$'|2.33        |3.1         |5.17        |3.31        |2.33        |2.9         |3.07         |
|$\psi$ |251.2       |251 **?**   |171         |167         |166         |149         |167 **?**    |

#### For Type II Aliased Mazes

|       |MazeF4      |Maze7       |Sutton      |Lab1        |MiyazakiB   |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|
|$d$    |7           |7           | 10         | **\***     |5           |
|$\phi$'|4.5 **?**   |4.33        | **\***     | **\***     |3.33        |
|$\psi$ |47 **?**    |82 **?**    | **?**      | **?**      |1.03        |

#### For Type I Aliased Mazes

|       |MazeB       |Littman89   |MiyazakiA   |MazeD       |Cassandra4x4|Littman57   |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$d$    |5           |7           |5           |4           |3           |8           |
|$\phi$'|3.5         |3.77        |3.05        |2.75        |2.27        |3.71        |
|$\psi$ |1.26        |61 **?**    |69 **?**    |1.03        |1           |154 **?**   |

#### For Non Aliased Mazes

|       |Maze4       |Maze5       |MazeA       |MazeF1      |MazeF2      |MazeF3      |Woods1      |Woods14     |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$d$    |5           |8           |6           |3           |4           |5           |2           |18          |
|$\phi$ |3.5         |4.61        |4.23        |1.8         |2.5         |3.38        |1.63        |9.5         |
|$\psi$ | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     |